In [5]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [6]:
import tensorflow as tf

def _inception_module(input_tensor, stride=1, activation='linear', use_bottleneck=True, kernel_size=40, bottleneck_size=32, nb_filters=32):

    if use_bottleneck and int(input_tensor.shape[-1]) > 1:
        input_inception = tf.keras.layers.Conv1D(filters=bottleneck_size, kernel_size=1,
                                              padding='same', activation=activation, use_bias=False)(input_tensor)
    else:
        input_inception = input_tensor

    # kernel_size_s = [3, 5, 8, 11, 17]
    kernel_size_s = [kernel_size // (2 ** i) for i in range(3)]

    conv_list = []

    for i in range(len(kernel_size_s)):
        conv_list.append(tf.keras.layers.Conv1D(filters=nb_filters, kernel_size=kernel_size_s[i],
                                              strides=stride, padding='same', activation=activation, use_bias=False)(
            input_inception))

    max_pool_1 = tf.keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

    conv_6 = tf.keras.layers.Conv1D(filters=nb_filters, kernel_size=1,
                                  padding='same', activation=activation, use_bias=False)(max_pool_1)

    conv_list.append(conv_6)

    x = tf.keras.layers.Concatenate(axis=2)(conv_list)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(activation='relu')(x)
    return x

def _shortcut_layer(input_tensor, out_tensor):
    shortcut_y = tf.keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                      padding='same', use_bias=False)(input_tensor)
    shortcut_y = tf.keras.layers.BatchNormalization()(shortcut_y)

    x = tf.keras.layers.Add()([shortcut_y, out_tensor])
    x = tf.keras.layers.Activation('relu')(x)
    return x

def build_iception_model(input_shape, num_classes, clf="binary", depth=6, use_residual=True, lr_init = 0.001, kernel_size=40, bottleneck_size=32, nb_filters=32):
    input_layer = tf.keras.layers.Input(input_shape)

    x = input_layer
    input_res = input_layer

    for d in range(depth):

        x = _inception_module(x,kernel_size = kernel_size, bottleneck_size=bottleneck_size, nb_filters=nb_filters)

        if use_residual and d % 3 == 2:
            x = _shortcut_layer(input_res, x)
            input_res = x

    gap_layer = tf.keras.layers.GlobalAveragePooling1D(name="feature_vector")(x)

 
    output_layer = tf.keras.layers.Dense(units = num_classes ,activation="sigmoid")(gap_layer)
    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr_init), loss = tf.keras.losses.BinaryFocalCrossentropy(), #loss = tf.keras.losses.CategoricalCrossentropy(), 
        metrics = [tf.keras.metrics.AUC(curve='ROC',name="AUROC", multi_label=True, num_labels=num_classes),tf.keras.metrics.AUC(curve='PR',name="AUPRC", multi_label=True, num_labels=num_classes)])
    
    print("Inception model built.")
    return model




In [7]:
model = build_iception_model((1000,12), 30)

Inception model built.


In [22]:
a = model.get_weights()

In [23]:
b = model.get_weights()

In [24]:
my_list = []

In [25]:
my_list.append(a)
my_list.append(b)

In [26]:
my_list = np.asarray(my_list)

C:\Users\Bjorn\AppData\Local\Temp\ipykernel_20736\443466448.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  my_list = np.asarray(my_list)


In [28]:
my_list.mean(axis=0).shape

(66,)

In [19]:
a = np.asarray(a)

C:\Users\Bjorn\AppData\Local\Temp\ipykernel_20736\658968879.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asarray(a)


In [20]:
b = np.asarray(b)

C:\Users\Bjorn\AppData\Local\Temp\ipykernel_20736\2925983279.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  b = np.asarray(b)


array([array([[[-4.83829081e-02,  1.54595822e-01,  2.67195970e-01,
                 2.44489342e-01,  3.10396522e-01,  2.27263600e-01,
                 1.24211520e-01, -5.67704439e-04, -3.45769525e-02,
                -4.59989011e-02, -1.28962800e-01,  2.26282090e-01,
                -4.02653515e-02, -2.82241702e-01,  2.05120146e-02,
                -2.49033004e-01,  2.22206146e-01,  3.50413889e-01,
                 2.14899987e-01, -2.20704079e-03,  3.66152614e-01,
                -2.55264342e-02, -1.28704131e-01,  3.27046663e-01,
                -9.87041295e-02,  2.07408577e-01,  1.48759872e-01,
                -2.15356812e-01, -1.47639647e-01,  2.30228037e-01,
                 3.31672817e-01, -3.66061032e-01],
               [ 3.64981443e-01,  2.24359244e-01, -4.33230400e-02,
                 1.01283669e-01,  4.39772010e-02,  1.14434391e-01,
                -2.37774566e-01, -2.75261492e-01,  2.81943589e-01,
                 2.10285932e-01, -3.13820750e-01,  1.96168721e-02,
           

In [15]:
model.set_weights(a)

In [36]:
a = np.load("../data/processed/y_data_stpeter.npy")

In [37]:
b = np.load("../data/processed/y_data_ptbdiag.npy")

In [38]:
np.vstack([a,b]).shape

(590, 30)